<a href="https://colab.research.google.com/github/0xVolt/whats-up-doc/blob/main/test/notebooks/model-blending/blend.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Merging CodeLLMs to Create an Efficant Low-Memory Quantized Model for `whats-up-doc` using the TIES Method

## Download and Install `mergekit`

In [1]:
import os

dirName = "mergekit"
cwd = os.getcwd()

concatDirPath = os.path.join(cwd, dirName)

if not os.path.exists(concatDirPath):
    !git clone https://github.com/cg123/mergekit.git
    !cd mergekit && pip install -q -e .

Cloning into 'mergekit'...
remote: Enumerating objects: 2265, done.
remote: Counting objects: 100% (1340/1340), done.
remote: Compressing objects: 100% (514/514), done.
remote: Total 2265 (delta 1068), reused 939 (delta 825), pack-reused 925
Receiving objects: 100% (2265/2265), 638.44 KiB | 16.80 MiB/s, done.
Resolving deltas: 100% (1584/1584), done.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
keras-cv 0.8.2 requires keras-core, which is not installed.
keras-nlp 0.9.3 requires keras-core, which is not installed.
beatrix-jupyterlab 2023.128.151533 requires jupyterlab~=3.6.0, but you have jupyterlab 4.1.6 which is incompatible.
momepy 0.7.0 requires shapely>=2, but you have shapely 1.8.5.post1 which is incompatible.
spopt 0.6.0 requires shapely>=2.0.1, but you have shapely 1.8.5.post1 which is incompatible.
ydata-profiling 4.6.4 requires numpy<1.26,>=1.16.0, 

In [16]:
from huggingface_hub import notebook_login
notebook_login()

## Create the YAML Config File to Merge Models with SLERP

In [17]:
import os
import yaml
from transformers import AutoModelWithLMHead, AutoTokenizer, pipeline

### Write Config Script

In [12]:
# Set the resultant model's name
MODEL_NAME = 'whats-up-llamas-ties'

MODEL_1 = "meta-llama/Meta-Llama-3-8B-Instruct"
MODEL_2 = "codellama/CodeLlama-7b-Instruct-hf"

OUTPUT_DIR = "whats-up-llamas-ties"

#### TIES YAML Config Creation

In [13]:
yamlConfigTIESLlamas = f"""
models:
  - model: {MODEL_1}  # no parameters necessary for base model
  - model: {MODEL_2}
    parameters:
      density: 0.5
      weight: 0.5
merge_method: ties
base_model: {MODEL_1}
parameters:
  normalize: true
  int8_mask: true
dtype: float16

"""

### Save Config Script

In [14]:
# Save the YAML configuration to a file
yamlFileName = "config.yaml"
with open(yamlFileName, "w") as f:
    f.write(yamlConfigTIESLlamas)

## Merge Models

In [18]:
cmd = f"mergekit-yaml {yamlFileName} {OUTPUT_DIR} --allow-crimes --copy-tokenizer --out-shard-size 1B --low-cpu-memory --write-model-card --lazy-unpickle"
!{cmd}

Fetching 10 files:   0%|                                 | 0/10 [00:00<?, ?it/s]

tokenizer.json:   0%|                               | 0.00/1.84M [00:00<?, ?B/s]


model-00002-of-00002.safetensors:   0%|             | 0.00/3.50G [00:00<?, ?B/s]



pytorch_model.bin.index.json: 100%|████████| 23.9k/23.9k [00:00<00:00, 68.2MB/s]




model.safetensors.index.json: 100%|████████| 25.1k/25.1k [00:00<00:00, 83.8MB/s]




special_tokens_map.json: 100%|█████████████████| 411/411 [00:00<00:00, 2.52MB/s]




generation_config.json: 100%|███████████████████| 116/116 [00:00<00:00, 544kB/s]

Fetching 10 files:  20%|█████                    | 2/10 [00:00<00:01,  6.49it/s]


model-00002-of-00002.safetensors:   0%|     | 10.5M/3.50G [00:00<00:34, 100MB/s]



model-00001-of-00002.safetensors:   0%|             | 0.00/9.98G [00:00<?, ?B/s]

tokenizer.json: 100%|██████████████████████| 1.84M/1.84M [00:00<00:00, 15.4MB/s]


tokenizer_config.json: 100%|███████████████| 1.59k/1.59k [00:00<00:00, 8.54MB/s]



In [19]:
from kaggle_secrets import UserSecretsClient
userSecrets = UserSecretsClient()
HF_WRITE_TOKEN = userSecrets.get_secret("HF_WRITE_TOKEN")

In [23]:
from huggingface_hub import HfApi

username = "0xVolt"

# Defined in the secrets tab in Kaggle Secrets
api = HfApi(token=HF_WRITE_TOKEN)

api.create_repo(
    repo_id=f"{username}/{MODEL_NAME}",
    repo_type="model"
)

# Push the whole merged-model folder to the hub
api.upload_folder(
    repo_id=f"{username}/{MODEL_NAME}",
    folder_path=OUTPUT_DIR,
)

HfHubHTTPError: 409 Client Error: Conflict for url: https://huggingface.co/api/repos/create (Request ID: Root=1-6649e7c6-04b8eadf19b91db250b52c67;8fb9e508-c301-48ed-9a29-1d25d0489d23)

You already created this model repo